# 1. Get variables

In [2]:
keyvaultlsname = 'Ls_KeyVault_01'
adls2lsname = 'Ls_AdlsGen2_01'

StatementMeta(synspdevdep70, 40, 2, Finished, Available)

# 2. Linked Services Setup: KV and ADLS Gen2

In [3]:
from pyspark.sql import SparkSession

sc = SparkSession.builder.getOrCreate()
token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary
storage_account = token_library.getSecretWithLS(keyvaultlsname, "datalakeaccountname")

spark.conf.set("spark.storage.synapse.linkedServiceName", adls2lsname)
spark.conf.set("fs.azure.account.oauth.provider.type", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")


StatementMeta(synspdevdep70, 40, 3, Finished, Available)

# 3. Create Schemas

In [9]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS dw LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS lnd LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS interim LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data'")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS malformed LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data'")

StatementMeta(synspdevdep70, 40, 9, Finished, Available)

DataFrame[]

# 4. Create Fact Tables

In [4]:
spark.sql(f"DROP TABLE IF EXISTS dw.fact_parking")

spark.sql(f"CREATE TABLE dw.fact_parking(dim_date_id STRING,dim_time_id STRING, dim_parking_bay_id STRING, dim_location_id STRING, dim_st_marker_id STRING, status STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/dw/fact_parking/'")
 
spark.sql(f"REFRESH TABLE dw.fact_parking")

StatementMeta(synspdevdep70, 40, 4, Finished, Available)

DataFrame[]

# 5. Create Dimension Tables

In [5]:
spark.sql(f"DROP TABLE IF EXISTS dw.dim_st_marker")
spark.sql(f"CREATE TABLE dw.dim_st_marker(dim_st_marker_id STRING, st_marker_id STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/dw/dim_st_marker/'")
spark.sql(f"REFRESH TABLE dw.dim_st_marker")
 

spark.sql(f"DROP TABLE IF EXISTS dw.dim_location")
spark.sql(f"CREATE TABLE dw.dim_location(dim_location_id STRING,lat FLOAT, lon FLOAT, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/dw/dim_location/'")
spark.sql(f"REFRESH TABLE dw.dim_location")
 

spark.sql(f"DROP TABLE IF EXISTS dw.dim_parking_bay")
spark.sql(f"CREATE TABLE dw.dim_parking_bay(dim_parking_bay_id STRING, bay_id INT,`marker_id` STRING, `meter_id` STRING, `rd_seg_dsc` STRING, `rd_seg_id` STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/dw/dim_parking_bay/'")
spark.sql(f"REFRESH TABLE dw.dim_parking_bay")

StatementMeta(synspdevdep70, 40, 5, Finished, Available)

DataFrame[]

# 6. Create dim date and time

In [6]:
from pyspark.sql.functions import col
spark.sql(f"DROP TABLE IF EXISTS dw.dim_date")
spark.sql(f"DROP TABLE IF EXISTS dw.dim_time")

# DimDate
dimdate = spark.read.csv(f"abfss://datalake@{storage_account}.dfs.core.windows.net/data/seed/dim_date/dim_date.csv", header=True)
dimdate.write.saveAsTable("dw.dim_date")

# DimTime
dimtime = spark.read.csv(f"abfss://datalake@{storage_account}.dfs.core.windows.net/data/seed/dim_time/dim_time.csv", header=True)
dimtime.write.saveAsTable("dw.dim_time")

StatementMeta(synspdevdep70, 40, 6, Finished, Available)

# 7. Create interim and error tables

In [8]:
spark.sql(f"DROP TABLE IF EXISTS interim.parking_bay")
spark.sql(f"CREATE TABLE interim.parking_bay(bay_id INT, `last_edit` TIMESTAMP, `marker_id` STRING, `meter_id` STRING, `rd_seg_dsc` STRING, `rd_seg_id` STRING, `the_geom` STRUCT<`coordinates`: ARRAY<ARRAY<ARRAY<ARRAY<DOUBLE>>>>, `type`: STRING>, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/interim/interim.parking_bay/'")
spark.sql(f"REFRESH TABLE interim.parking_bay")

spark.sql(f"DROP TABLE IF EXISTS interim.sensor")
spark.sql(f"CREATE TABLE  interim.sensor(bay_id INT, `st_marker_id` STRING, `lat` FLOAT, `lon` FLOAT, `location` STRUCT<`coordinates`: ARRAY<DOUBLE>, `type`: STRING>, `status` STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/interim/interim.sensor/'")
spark.sql(f"REFRESH TABLE  interim.sensor")
   

spark.sql(f"DROP TABLE IF EXISTS malformed.parking_bay")
spark.sql(f"CREATE TABLE malformed.parking_bay(bay_id INT, `last_edit` TIMESTAMP,`marker_id` STRING, `meter_id` STRING, `rd_seg_dsc` STRING, `rd_seg_id` STRING, `the_geom` STRUCT<`coordinates`: ARRAY<ARRAY<ARRAY<ARRAY<DOUBLE>>>>, `type`: STRING>, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/malformed/malformed.parking_bay/'")
spark.sql(f"REFRESH TABLE malformed.parking_bay")

spark.sql(f"DROP TABLE IF EXISTS malformed.sensor")
spark.sql(f"CREATE TABLE malformed.sensor(bay_id INT,`st_marker_id` STRING,`lat` FLOAT,`lon` FLOAT,`location` STRUCT<`coordinates`: ARRAY<DOUBLE>, `type`: STRING>,`status` STRING, load_id STRING, loaded_on TIMESTAMP) USING parquet LOCATION 'abfss://datalake@{storage_account}.dfs.core.windows.net/data/malformed/malformed.parking_bay/'")
spark.sql(f"REFRESH TABLE malformed.sensor")

StatementMeta(synspdevdep70, 40, 8, Finished, Available)

DataFrame[]